In [45]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [7]:
class splitData():
    def __init__(self):
        self.X_train = []
        self.X_test = []
        self.y_train = []
        self.y_test = []
    
    def split(self, x, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [21]:
features = ['district_encoder',\
        'area',\
        'new_num_floors',\
        'new_bedrooms',\
        'houseTypes_Bán Luxury home',\
        'houseTypes_Bán Nhà',\
        'houseTypes_Bán Nhà cổ',\
        'houseTypes_Bán Nhà mặt phố',\
        'houseTypes_Bán Nhà riêng']

df = pd.read_excel('HCM_data.xlsx')
df_tranform = pd.DataFrame(data = StandardScaler().fit_transform(df.loc[:, features].values), columns = features)
y = df['price'].values
x = df_tranform[features].values
data = splitData()
data.split(x, y)

# RNN seq

## simple ANN


In [209]:

class ModelRNN(keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = keras.layers.Dense(20, activation='relu'), #fully connected networks
        self.dense2 = keras.layers.Dense(20, activation='relu'),
        self.dense3 = keras.layers.Dense(20, activation='relu'),
        self.dense4 = keras.layers.Dense(1)
        
        self.model = Sequential([self.dense1, self.dense2, self.dense3, self.dense4])
        
    def call(self,inputs):
        return self.model(inputs)
    
    def compile(self, optimizer="adam", loss="mse"):
        super().compile(optimizer, loss)
        
    def fit(self, data, epochs = 500, batch_size=128):
        super().fit(data.X_train, data.y_train, epochs, batch_size)
        
    def evaluation(self, data):
        # Predict the values for the test dataset.
        y_pred = self.predict(data.X_test)

        # Calculate the MSE, MAE, and explained variance score.
        from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
        mse = mean_squared_error(data.y_test, y_pred)
        mae = mean_absolute_error(data.y_test, y_pred)
        evs = explained_variance_score(data.y_test, y_pred)

        # Print the evaluation metrics.
        print("MSE: ", mse)
        print("MAE: ", mae)
        print("variance: ", evs)

In [210]:
model_rnn = ModelRNN()
model_rnn.compile(optimizer='adam', loss='mse')
model_rnn.fit(data, epochs=500, batch_size =128)
model_rnn.evaluation(data)

TypeError: The added layer must be an instance of class Layer. Received: layer=(<keras.layers.core.dense.Dense object at 0x000001FFFF5B5D50>,) of type <class 'tensorflow.python.trackable.data_structures._TupleWrapper'>.

## Tranformer


In [181]:
class TransformerModel(keras.Model):
    
    def __init__(self):
        super().__init__()

        self.encoder = Sequential([
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(64, activation='relu'),
        ])

        self.decoder = Sequential([
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(1)
        ])
        
    def call(self, inputs):
        encoder_outputs = self.encoder(inputs)
        decoder_outputs = self.decoder(encoder_outputs)
        return decoder_outputs
    
    def compile(self, optimizer='adam', loss='mse'):
        super().compile(optimizer, loss)

    def fit(self, x_train, y_train, epochs=500, batch_size =128):
        super().fit(x_train, y_train, epochs, batch_size)
    




In [182]:
model = TransformerModel()
model.compile(optimizer='adam', loss='mse')
model.fit(data.X_train, data.y_train, epochs=500, batch_size =128)


Epoch 1/128
9/9 [==============================] - 1s 5ms/step - loss: 527.2480
Epoch 2/128
9/9 [==============================] - 0s 7ms/step - loss: 463.7621
Epoch 3/128
9/9 [==============================] - 0s 5ms/step - loss: 431.2480
Epoch 4/128
9/9 [==============================] - 0s 6ms/step - loss: 416.6245
Epoch 5/128
9/9 [==============================] - 0s 6ms/step - loss: 410.2328
Epoch 6/128
9/9 [==============================] - 0s 7ms/step - loss: 407.3193
Epoch 7/128
9/9 [==============================] - 0s 6ms/step - loss: 398.8887
Epoch 8/128
9/9 [==============================] - 0s 7ms/step - loss: 396.1284
Epoch 9/128
9/9 [==============================] - 0s 6ms/step - loss: 392.0272
Epoch 10/128
9/9 [==============================] - 0s 6ms/step - loss: 390.1367
Epoch 11/128
9/9 [==============================] - 0s 6ms/step - loss: 383.0321
Epoch 12/128
9/9 [==============================] - 0s 6ms/step - loss: 378.8950
Epoch 13/128
9/9 [===================

In [40]:
import tensorflow as tf
vocab_size = 256

class Transformer(tf.keras.Model):

    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(Transformer, self).__init__()

        self.encoder = TransformerEncoder(num_layers, d_model, num_heads, dropout_rate)
        self.decoder = TransformerDecoder(num_layers, d_model, num_heads, dropout_rate)

    def call(self, inputs, training=False):
        encoder_outputs = self.encoder(inputs, training=training)
        decoder_outputs = self.decoder(encoder_outputs, training=training)

        return decoder_outputs

class TransformerEncoder(tf.keras.layers.Layer):

    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(TransformerEncoder, self).__init__()

        self.attention_layers = []
        for _ in range(num_layers):
            self.attention_layers.append(
                tf.keras.layers.MultiHeadAttention(
                    num_heads=num_heads,
                    d_model=d_model,
                    dropout_rate=dropout_rate
                )
            )
        self.feed_forward_layers = []
        for _ in range(num_layers):
            self.feed_forward_layers.append(
                tf.keras.layers.Dense(
                    d_model,
                    activation="relu",
                    dropout_rate=dropout_rate
                )
            )

    def call(self, inputs, training=False):
        for attention_layer in self.attention_layers:
            inputs = attention_layer(inputs, inputs, training=training)
            inputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)

        for feed_forward_layer in self.feed_forward_layers:
            inputs = feed_forward_layer(inputs)
            inputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)

        return inputs

class TransformerDecoder(tf.keras.layers.Layer):

    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(TransformerDecoder, self).__init__()

        self.attention_layers = []
        for _ in range(num_layers):
            self.attention_layers.append(
                tf.keras.layers.MultiHeadAttention(
                    num_heads=num_heads,
                    d_model=d_model,
                    dropout_rate=dropout_rate
                )
            )
        self.feed_forward_layers = []
        for _ in range(num_layers):
            self.feed_forward_layers.append(
                tf.keras.layers.Dense(
                    d_model,
                    activation="relu",
                    dropout_rate=dropout_rate
                )
            )

        self.final_layer = tf.keras.layers.Dense(
            vocab_size,
            activation="softmax"
        )

    def call(self, inputs, encoder_outputs, training=False):
        for attention_layer in self.attention_layers:
            inputs = attention_layer(inputs, encoder_outputs, training=training)
            inputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)

        for feed_forward_layer in self.feed_forward_layers:
            inputs = feed_forward_layer(inputs)
            inputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)

        outputs = self.final_layer(inputs)

        return outputs